#Lab 1: Introduction to Big Data Processing




## Introduction
Big data processing involves techniques to efficiently handle and analyze datasets too large to fit into memory. This lab focuses on understanding partitioning, aggregation, sorting, and distributed systems concepts foundational to tools like MapReduce, Hadoop, and Spark.

##Helper: Timed Decorator
To evaluate the execution time of each function systematically, we can create a reusable timed decorator.
The decorator logs the execution time of any function it wraps.

In [1]:
import time
from functools import wraps

function_perf_tracker = {}

def timed(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Function '{func.__name__}' executed in {end_time - start_time:.4f} seconds.")
        function_perf_tracker[func.__name__] = end_time - start_time
        return result
    return wrapper


##Exercise 1: Searching via Partitioning
Efficiently search for integers in a large dataset using both naive (linear) and optimized (partitioned) approaches.



###Step 1: Dataset Generation

Use Python's random module to generate the dataset:

In [2]:
import random

# Generate dataset
with open("search_data.txt", "w") as file:
    for _ in range(900000):
        file.write(f"{random.randint(1, 1000000)}\n")


###Step 2: Linear Search

A naive approach to scan the dataset sequentially:

In [4]:
@timed
def linear_search(filename, targets):
    # Convert targets to a set for efficient membership checks
    target_set = set(targets)
    results = {target: "NO" for target in targets}  # Initialize results as "NO"

    with open(filename, "r") as file:
        for line in file:
            num = int(line.strip())
            if num in target_set:
                results[num] = "YES"

    # Return the results in the same order as the input targets
    return [results[target] for target in targets]

# Example usage
targets = [5, 1000, 250000, 750000, 999999]
print(linear_search("search_data.txt", targets))


Function 'linear_search' executed in 0.2944 seconds.
['NO', 'YES', 'NO', 'YES', 'YES']


### Step 3: Partitioned Search

Partition the dataset into 1,000 smaller files:

In [5]:
def partition_dataset(input_file, partitions=1000):
    partition_files = [open(f"partition_{i}.txt", "w") for i in range(partitions)]

    with open(input_file, "r") as file:
        for line in file:
            num = int(line.strip())
            partition_files[num % partitions].write(line)

    for pf in partition_files:
        pf.close()

partition_dataset("search_data.txt")

**Search in the relevant partition file:**

In [6]:
@timed
def partitioned_search(partitions, targets):
    target_set = set(targets)
    results = {target: "NO" for target in targets}  # Initialize results as "NO"
    keys = set([n%partitions for n in targets])
    for key in keys:
        partition_file = f"partition_{key}.txt"
        with open(partition_file, "r") as file:
            for line in file:
              num = int(line.strip())
              if num in target_set:
                  results[num] = "YES"
                  target_set.remove(num)

    return [results[target] for target in targets]

print(partitioned_search(1000, targets))

Function 'partitioned_search' executed in 0.0017 seconds.
['NO', 'YES', 'NO', 'YES', 'YES']


###Key Takeaway
Partitioning significantly reduces the search space, mimicking distributed file systems' efficiency.

##Exercise 2: Grouping and Aggregation

Aggregate data by grouping keys, using naive and partitioned methods.

###Step 1: Dataset Generation


In [8]:
# Generate dataset
with open("group_data.txt", "w") as file:
    for _ in range(30000000):
        k = random.randint(1, 7)
        v = random.randint(1, 1000)
        file.write(f"{k},{v}\n")

###Step 2: Naive Grouping

Read the file and compute aggregation:

In [10]:
from collections import defaultdict

@timed
def naive_grouping(filename):
    aggregation = defaultdict(int)
    with open(filename, "r") as file:
        for line in file:
            k, v = map(int, line.strip().split(","))
            aggregation[k] += v
    return sorted(aggregation.items())

print(naive_grouping("group_data.txt"))


Function 'naive_grouping' executed in 27.9796 seconds.
[(1, 2148384856), (2, 2144520357), (3, 2140954403), (4, 2143625377), (5, 2145918822), (6, 2147204075), (7, 2145779744)]


###Step 3: Partitioned Grouping

**Partition the dataset:**


In [13]:
def partition_group_data(input_file, partitions=10):
    partition_files = [open(f"group_partition_{i}.txt", "w") for i in range(partitions)]

    with open(input_file, "r") as file:
        for line in file:
            k, v = map(int, line.strip().split(","))
            partition_files[v % partitions].write(line)

    for pf in partition_files:
        pf.close()

partition_group_data("group_data.txt", 100)

**Aggregate**

In [14]:
from collections import defaultdict

@timed
def partitioned_grouping(partitions=10):
    final_aggregation = defaultdict(int)
    all_local_aggregations = []

    # Compute local aggregations
    for i in range(partitions):
        local_aggregation = defaultdict(int)
        with open(f"group_partition_{i}.txt", "r") as file:
            for line in file:
                k, v = map(int, line.strip().split(","))
                local_aggregation[k] += v

        all_local_aggregations.append(local_aggregation)

    # Merge local aggregations
    for local_aggregation in all_local_aggregations:
      for k, s in local_aggregation.items():
          final_aggregation[k] += s

    return final_aggregation

aggregation_sum = partitioned_grouping(100)
print(sorted(aggregation_sum.items()))


Function 'partitioned_grouping' executed in 28.7355 seconds.
[(1, 2146449325), (2, 2144686253), (3, 2143693767), (4, 2143779031), (5, 2144550492), (6, 2145944290), (7, 2144219893)]


###Key Takeaway
Partitioning simulates distributed "reduce" operations, optimizing performance for large-scale data.

## Exercise 2bis: Sorting and Grouping by Key

Perform grouping and aggregation on a sorted dataset by key, simulating how sorting helps optimize grouping operations in big data systems.

###Step 1: Dataset Generation
Generate the sorted dataset

In [ ]:
import random


def generate_sorted_group_data(file_path, size=3000000):
    """
    Generate a dataset of (k, v) pairs and save it sorted by k.

    Args:
        file_path (str): File to save the dataset.
        size (int): Number of (k, v) pairs to generate.
    """
    dataset = [(random.randint(1, 7), random.randint(1, 1000)) for _ in range(size)]
    dataset.sort(key=lambda x: x[0])  # Sort by k
    with open(file_path, "w") as file:
        file.writelines(f"{k},{v}\n" for k, v in dataset)
    print(f"Generated sorted dataset and saved to {file_path}.")

generate_sorted_group_data("sorted_group_data.txt")

Generated sorted dataset and saved to sorted_group_data.txt.


### Step2:  Iterator-based Grouping

Group and sum values by key using an iterator over a sorted dataset.


In [ ]:
from itertools import groupby

@timed
def iterator_based_grouping(file_path):
    """
    Group and sum values by key using an iterator over a sorted dataset.

    Args:
        file_path (str): Path to the sorted dataset.

    Returns:
        dict: Aggregated sum of values for each key.
    """
    aggregation = {}
    with open(file_path, "r") as file:
        # Group consecutive lines with the same key using groupby
        for k, group in groupby(file, key=lambda line: int(line.split(",")[0])):
            aggregation[k] = sum(int(line.split(",")[1]) for line in group)
    return aggregation

sorted_aggregation_data = iterator_based_grouping("sorted_group_data.txt")
sorted_aggregation_data = sorted(sorted_aggregation_data.items())
print(sorted_aggregation_data)

Function 'iterator_based_grouping' executed in 3.0983 seconds.
[(1, 450146929), (2, 300350528), (3, 300587224), (4, 150624761), (5, 149503408), (6, 74968250), (7, 75078409)]


### Step 3: Grouping via Iteration
Implement a function to iterate through the sorted dataset, grouping values by \( k \) as they appear.

In [ ]:
@timed
def grouping_by_iteration(file_path):
    aggregation = {}
    current_key = None
    current_sum = 0
    with open(file_path, "r") as file:
        for line in file:
            k, v = map(int, line.strip().split(","))
            if k != current_key:
                if current_key is not None:
                    aggregation[current_key] = current_sum
                current_key, current_sum = k, v
            else:
                current_sum += v
    return aggregation

aggregation_data = grouping_by_iteration("sorted_group_data.txt")
aggregation_data = sorted(aggregation_data.items())
print(aggregation_data)

Function 'grouping_by_iteration' executed in 2.3080 seconds.
[(1, 450146929), (2, 300350528), (3, 300587224), (4, 150624761), (5, 149503408), (6, 74968250)]


### Discussion
- Measure execution time for:
  1. **Naive Grouping**: Reads and groups an unsorted file by scanning and aggregating in memory.
  2. **Iterator-based Grouping**: Processes the sorted file line by line using the grouping-by-iteration method.
- Compare the performance of the iterator-based grouping on the sorted file against the naive grouping on an unsorted dataset.

##Exercise 3: n-way Merge-Sort


###Step 1: Dataset Preparation
Generate and save the sorted lists as in the original exercise.


In [16]:
import random

# Generate sorted lists and save to files
lists = [
    sorted(random.randint(1, 100) for _ in range(10)),
    sorted(random.randint(50, 150) for _ in range(10)),
    sorted(random.randint(100, 200) for _ in range(10))
]

for i, lst in enumerate(lists):
    with open(f"list_{i}.txt", "w") as file:
        file.writelines(f"{x}\n" for x in lst)


###Step 2: N-way Merge-Sort Implementation
The idea is to read numbers from all files sequentially, maintaining a pointer (or current position) in each file to track which number should be considered next.

In [20]:
@timed
def n_way_merge(files):
    # Open all files
    open_files = [open(file, "r") for file in files]

    # Read the first line from each file to initialize the pointers
    pointers = [int(file.readline().strip()) if not file.closed else None for file in open_files]

    output_filename = "sorted_output.txt"

    # Merge the files
    with open(output_filename, "w") as output_file:
      while any(pointers):  # Continue while at least one file still has data
          # Find the smallest value among the current pointers
          min_value = min(value for value in pointers if value is not None)
          output_file.write(f"{min_value}\n")

          # Update the pointer for the file from which the min_value came
          min_value_index = pointers.index(min_value)
          next_line = open_files[min_value_index].readline().strip()
          pointers[min_value_index] = int(next_line) if next_line else None

    # Close all files
    for file in open_files:
        file.close()

    return output_filename

# Example usage
files = [f"list_{i}.txt" for i in range(3)]

merged_output_file = n_way_merge(files)
with open(merged_output_file, "r") as file:
    merged_output = map(int, file.read().strip().split("\n"))
    merged_output = list(merged_output)

print(merged_output)


Function 'n_way_merge' executed in 0.0005 seconds.
[2, 4, 18, 45, 45, 54, 60, 60, 85, 85, 87, 100, 105, 106, 107, 110, 119, 121, 124, 138, 141, 144, 144, 145, 145, 149, 161, 187, 192, 193]


**How It Works**
1. **Initialization:**

    - Open all files and read the first line from each file to initialize the pointers.
    - Store these first values in a pointers list.
2. **Find the Smallest Value:**

    - Use the built-in min() function to find the smallest value in the current pointers.
3. **Update Pointers:**

    - Determine which file contributed the smallest value and update its pointer by reading the next line from that file.
4. **Repeat:**

    - Continue until all files are fully read and no values remain in the pointers list.
5. **Write Output:**

    - Store the merged values in a list or write them directly to a file.

##Exercise 4: Word Counting
Count word occurrences in a large text dataset using both naive (sequential) and partitioned (distributed) methods. This exercise simulates MapReduce-style word counting.


###Step 1: Dataset Preparation
Prepare a large text dataset (text_data.txt). For simplicity, let's create a file with random sentences.

In [21]:
import random
import string

# Generate random sentences
def generate_text_file(filename, num_lines=1000000):
    words = ["apple", "banana", "orange", "grape", "pineapple", "kiwi", "melon"]
    with open(filename, "w") as file:
        for _ in range(num_lines):
            line = " ".join(random.choices(words, k=random.randint(5, 15)))
            file.write(f"{line}\n")

generate_text_file(f"text_data.txt")

###Step 2: Sequential Word Count
Count word occurrences by scanning the file line by line.

In [22]:
from collections import defaultdict

@timed
def sequential_word_count(filename):
    word_counts = defaultdict(int)
    with open(filename, "r") as file:
        for line in file:
            for word in line.strip().split():
                word_counts[word] += 1
    return word_counts

# Example usage
word_counts = sequential_word_count("text_data.txt")
word_counts = sorted(word_counts.items())
print(word_counts[:10])  # Print top 10 word counts


Function 'sequential_word_count' executed in 2.6121 seconds.
[('apple', 1429847), ('banana', 1425329), ('grape', 1427017), ('kiwi', 1431207), ('melon', 1428040), ('orange', 1428904), ('pineapple', 1426944)]


###Step 3: Partitioned Word Count
Use a hash function to divide the dataset into smaller files, process each partition, and combine results.


**Partition the Dataset**
Partition the dataset into smaller files based on a hash function.

In [23]:
def partition_hash(key, num_partitions):
    return sum(ord(c) for c in key) % num_partitions


def partition_text_file(input_file, partitions=10):
    partition_files = [open(f"text_partition_{i}.txt", "w") for i in range(partitions)]
    with open(input_file, "r") as file:
        for line in file:
            words = line.strip().split()
            for word in words:
                partition_index = partition_hash(word, partitions)
                partition_files[partition_index].write(f"{word}\n")
    for pf in partition_files:
        pf.close()


partition_text_file("text_data.txt")


**Count Words in Each Partition**
Count word occurrences in each partition file.

In [24]:
def count_partition_file(filename):
    word_counts = defaultdict(int)
    with open(filename, "r") as file:
        for line in file:
            word_counts[line.strip()] += 1
    return word_counts

**Combine Results**
Aggregate word counts from all partitions.

In [25]:
@timed
def partitioned_word_count(partitions=10):
    partition_counts = []
    # Compute all local word count
    for i in range(partitions):  # can be parallized
        local_count = count_partition_file(f"text_partition_{i}.txt")
        partition_counts.append(local_count)

    # Aggregate the local word count
    combined_counts = defaultdict(int)
    for partition in partition_counts:
        for word, count in partition.items():
            combined_counts[word] += count

    return combined_counts

# Example usage
partitioned_counts = partitioned_word_count(10)
partitioned_counts = sorted(partitioned_counts.items())
print(partitioned_counts[:10])  # Print top 10 word counts


Function 'partitioned_word_count' executed in 2.7242 seconds.
[('apple', 1429847), ('banana', 1425329), ('grape', 1427017), ('kiwi', 1431207), ('melon', 1428040), ('orange', 1428904), ('pineapple', 1426944)]


###Discussion
**Sequential Approach:** Processes the entire dataset in one pass but can be slow for very large datasets due to memory constraints.  
**Partitioned Approach:** Divides work across multiple files, simulating parallel processing and reducing memory usage. This approach is scalable and forms the basis of MapReduce-style word counting.


###Key Takeaway
The partitioned approach is more scalable for large datasets, demonstrating how the "map" and "reduce" steps in distributed frameworks like Hadoop and Spark optimize big data processing.

## **Detailed Analysis**

1. **Exercise 1: Searching**
   - **Naive Approach**: Sequentially scans the entire file for each search query, which scales poorly as \(n\) (number of integers) grows.
   - **Partitioned Approach**: Limits the search to a smaller subset of the data by hashing, improving performance as \(m\) (number of partitions) increases.

2. **Exercise 2: Grouping**
   - **Naive Approach**: Directly aggregates values for each key in a single pass.
   - **Partitioned Approach**: Divides the dataset into \(m\) smaller groups, reducing memory overhead and simulating distributed parallel processing.

3. **Exercise 3: Merge-Sort**
   - **Pointer-Based Merge**: Simpler but less efficient, as each merge step compares all \(k\) current elements.
   - **`heapq` Merge**: Maintains a min-heap to quickly find the smallest element, reducing comparison overhead.

4. **Exercise 4: Word Count**
   - **Naive Approach**: Reads the dataset sequentially and counts words in memory, which becomes slow for very large datasets.
   - **Partitioned Approach**: Uses hashing to divide data into manageable chunks, allowing efficient in-memory counting for each partition.



## **Takeaways**
- **Partitioning**: Improves scalability in Exercises 1, 2, and 4 by reducing the size of the data each step processes.
- **Parallelism**: Many "advanced" methods simulate distributed systems, which are inherently more scalable for big data problems.


## Performance Track

In [ ]:
for key, value in function_perf_tracker.items():
  print(f"{key}: {value}")

linear_search: 0.6784422397613525
partitioned_search: 0.0029621124267578125
naive_grouping: 28.416772603988647
partitioned_grouping: 28.62277603149414
iterator_based_grouping: 3.09828782081604
grouping_by_iteration: 2.3079564571380615
n_way_merge: 0.0011546611785888672
sequential_word_count: 4.4674296379089355
partitioned_word_count: 4.637244701385498
